Done:
- allow easier python specdefinition
- %reg syntax for cool inline asm demos
- 


syscalls?


Interrupt preludes
It got a lot slower?
pi pico
More examples from frama-c, s2n-bignum
simd
reads/writes
kd_always. Some interrupt bug?

Change register names to enable true inline assembly.

Can I use unicorn to validate against that easily?


spacer? I'd be shocked if useful.



sizex <= x - y <= -sizey \/  sizey <= y - x <= -sizex

We can say exactly when the or happens
if x > y 
then
    sizey <= x - y <= -sizex
else
    sizey <= y - x <= -sizex

inequality and addition / diff son't play the same as the ints.

https://mcyoung.xyz/2023/11/27/simd-base64/  simd

Would I want to state from particular entry? or even address

kd_reachable from_label here_label
kd_reachable here_label
kd_unreachable from_label here_label

address expressions. Literal address for label, or `<foo+0x7>` relative addressing. I guess not necessary if this is mainly for textual assembly we have no control over. In that case just use programmatic bindings

https://github.com/toolCHAINZ/jingle


https://github.com/m4b/goblin 
https://github.com/m4b/bingrep
https://github.com/m4b/faerie


https://ruor.uottawa.ca/server/api/core/bitstreams/86cc22d7-4517-4744-9cf7-4b1745a6c029/content race conditions and memory models. AMusing because that _is_ what sail is for. Could be fun.

In [ ]:
kd_prelude "(define-fun nonoverlap ((a (_ BitVec 64)) (sa (_ BitVec 64)) (b (_ BitVec 64)) (sb (_ BitVec 64))))"
kd_prelude "(define-const bvone (_ bv1 64))"
kd_prelude "(define-const bvtwo (_ bv2 64))"
kd_prelude "(define-fun select32 ...)"

In [18]:
from kdrag.all import *

x,y = smt.BitVecs("x y", 64)
sizex, sizey = smt.Ints("sizex, sizey")
bsizex = smt.Int2BV(sizex, 64)
bsizey = smt.Int2BV(sizey, 64)
maxint = smt.BitVecVal(2**64-1, 64)
no_overlap = kd.define("no_overlap", [x,sizex,y,sizey], 
    smt.If( smt.ULE(x,y),
       smt.And( smt.ULT(bsizex , y - x), smt.ULE(bsizey, maxint - (y - x))),
       smt.And( smt.ULT(bsizey , x - y), smt.ULE(bsizex, maxint - (x - y)))
  ))

kd.prove(smt.Implies(smt.Distinct(x, x+1, y, y+1, y+2), no_overlap(x,2,y,3)), unfold=1)



LemmaError: (Implies(Distinct(x, x + 1, y, y + 1, y + 2),
        no_overlap(x, 2, y, 3)), 'Countermodel', [y = 18446744073709551613,
 x = 18446744073709551611,
 no_overlap = [else -> False]])

In [17]:
2**64-1

18446744073709551615

# ghost call
Figuring out how to access members of structs, offsets etc.
Shouldn't change state, can read it. Changed real state is thrown away (stored as var?). Changed ghost state is not.
How to setup frame?
More like a subroutine, since we odn't setup a stack (?).
In ghost code, read from named coroutine params.
Or give fresh
Know the abi somehow?

```
struct biz
get param(&biz) {
    return biz->a->b->c;
}
```

My dishonesty about ram32 ram64 rax bites now?
(RAX reg) (RDI reg)
%rdi == (select32 reg RDI)
(store32 reg RDI 0x42)
`kd_reify "memstate_name"` # freeze current state into ghost.
`kd_assign "memstate" "(store RAX  )"
`kd_ghost_call "res_memstate" "myfun" " "`
`kd_assign  "foo"  "res_memstate"

# read write
I should unpack the state datatype. No reason to bring datatypes into it.
Could use store_view instead of explciit write field?
Jankier

# verilog / sail
Pcode is not that trustworthy. Vet against other semantics.
(And why not just use those semantics instead ?)


https://docs.pydrofoil.org/en/latest/scripting-api.html

rpython
https://mssun.github.io/rpython-by-example/
So rpython itself is a bit like cython? But not exporting python libs. Using rpython on it's own like this is perhaps odd


smtlib interpreter in rpython?
Lean expr interpreter?



In [1]:
! nix build github:rpypkgs/rpypkgs#pydrofoil-riscv

34;1m1/18/19 built, 159 copied (1102.1/1102.7 MiB), 270.7 MiB DL] building pydrofoilm

In [3]:
! nix run github:rpypkgs/rpypkgs#pydrofoil-riscv -- --version

pydrofoil-riscv 0.0.1-alpha0 (Sail model version: 0.5-273-g87f8bb34d6)


In [ ]:
%%file /tmp/hello.py

def entry_point(argv):
    print "Hello, World!"
    return 0

# The target function is the main function of a RPython program. It takes
# command line arguments as inputs and return an entry point function.
def target(*args):
    return entry_point

# python properties


# Inline asm Reg names
Also memory locations?



In [4]:
import z3
z3.parse_smt2_string(r"""
(declare-const %rdi (_ BitVec 64))
(assert (= %rdi (_ bv0 64)))
""")


[%rdi == 0]

In [ ]:
%%file /tmp/hello.c
#include <stddef.h>
__asm__(".include \"knuckle.S\"");
__asm__("kd_prelude \"(declare-const x (_ BitVec 64))\"");
int foo(int x){
    __asm__("kd_entry myentry \"true\"");
    __asm__("kd_assign x %0" : : "r"(x));

    int y = x + 42;
    __asm__("kd_exit myentry \"(assert (= %0 (bvadd x (_ bv42 64)))\"" : : "r"(y));
    return y;
}


Overwriting /tmp/hello.c


In [9]:
! gcc -O1 -S -c /tmp/hello.c -o /tmp/hello.S && cat /tmp/hello.S

	.file	"hello.c"
	.text
	.globl	foo
	.type	foo, @function
foo:
.LFB0:
	.cfi_startproc
	endbr64
#APP
# 3 "/tmp/hello.c" 1
	kd_prelude "(assert (= %edi 42))"
# 0 "" 2
#NO_APP
	ret
	.cfi_endproc
.LFE0:
	.size	foo, .-foo
	.ident	"GCC: (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


# CN
https://github.com/rems-project/cn
https://rems-project.github.io/cn-tutorial/getting-started/tutorials/welcome/
https://www.galois.com/articles/escaping-isla-nublar-coming-around-to-llms-for-formal-methods

# cfg reconstruction

Takes VCs

Any unreached specstmt, warn
reachable_from

Join specstmts

Print CFG as ascii?
Output it?



In [ ]:

reachable = set().union(*[vc.trace for vc in vcs])

# Hyperproperties / Relational

Synchro points?

entry exit labels pairs.
kd_sync

kd_rel entry exit property
kd_rel entry_exit property

kd_channels
monotonic channels?

Finely interleaved product programs

A syncrhonzination / scheudling program?
ram_2

Multiram executor? Channels

Same program twice is easy to synchrnoize.
High low variables

havoc statement
kd_havoc_ghost ghost_var
kd_havoc_reg reg
kd_havoc_mem memexpr  
interrupt havocs touched stuff every cycle. So we do want to enable havor

https://docs.certora.com/en/latest/docs/cvl/statements.html

kd_join
collect up all trace fragements with common start and ends.
Maybe I should anyhow?

Useful for the using assembly as spec idea.

trace fragment bundle. Yikes

thread_id as a way of knowing which program you're in?





# Unicorn
Running stuff in pcode2c was clutch.
What about just a nice running environemnt that ingests asmspec.
Unicorn, gdb, qemu. Something.



